####  Model Fine-tuning
 Fine-tuning the Meta Llama 2 7B large language model, deploying the fine-tuned model, and testing it's text generation and domain knowledge capabilities. 

The number of files under train and validation should equal to one.
Output: A trained model that can be deployed for inference.

---

#### Set up


In [ ]:
!pip install --upgrade sagemaker datasets

Select the model to fine-tune

In [ ]:
model_id, model_version = "meta-textgeneration-llama-2-7b", "2.*"


To create a finance domain expert model: 

- `"training": f"s3://genaiwithawsproject2024/training-datasets/finance"`

To create a medical domain expert model: 

- `"training": f"s3://genaiwithawsproject2024/training-datasets/medical"`

To create an IT domain expert model: 

- `"training": f"s3://genaiwithawsproject2024/training-datasets/it"`

In [ ]:
from sagemaker.jumpstart.estimator import JumpStartEstimator
import boto3

estimator = JumpStartEstimator(model_id=model_id,  environment={"accept_eula": "true"},instance_type = "ml.g5.2xlarge") 

estimator.set_hyperparameters(instruction_tuned="False", epoch="5")

estimator.fit({"training": f"s3://genaiwithawsproject2024/training-datasets/finance"})


#### Deploy the fine-tuned model
---
Next, we deploy the domain fine-tuned model. We will compare the performance of the fine-tuned and pre-trained model.

---

In [ ]:

finetuned_predictor = estimator.deploy(instance_type="ml.g5.2xlarge", initial_instance_count=1)

#### Evaluate the pre-trained and fine-tuned model
---
Next, we use the same input from the model evaluation step to evaluate the performance of the fine-tuned model and compare it with the base pre-trained model. 

---

Create a function to print the response from the model

In [ ]:
def print_response(payload, response):
    print(payload["inputs"])
    print(f"> {response}")
    print("\n==================================\n")

Now we can run the same prompts on the fine-tuned model to evaluate it's domain knowledge.   


**For financial domain:**

- "The  investment  tests  performed  indicate"
- "the  relative  volume  for  the  long  out  of  the  money  options, indicates"
- "The  results  for  the  short  in  the  money  options"
- "The  results  are  encouraging  for  aggressive  investors"

**For medical domain:** 

- "Myeloid neoplasms and acute leukemias derive from"
- "Genomic characterization is essential for"
- "Certain germline disorders may be associated with"
- "In contrast to targeted approaches, genome-wide sequencing"

**For IT domain:** 

- "Traditional approaches to data management such as"
- "A second important aspect of ubiquitous computing environments is"
- "because ubiquitous computing is intended to" 
- "outline the key aspects of ubiquitous computing from a data management perspective."

In [ ]:
payload = {
    "inputs": "Traditional approaches to data management such as",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = finetuned_predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

In [ ]:
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()